In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow Cython matplotlib
!pip install simple-elmo
!pip install git+https://github.com/lopuhin/python-adagram.git
!pip install pymystem3 pymorphy2
!pip install wiktionary-parser-ru
!pip install corus

In [3]:
# библиотеки для работы с эмбеддингами
import adagram
from simple_elmo import ElmoModel

# парсинг
from wiktionaryparserru.parser import WiktionaryParser

# обработка данных и ML
import pandas as pd
from lxml import html
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from tqdm.notebook import tqdm
from sklearn.metrics import adjusted_rand_score
from sklearn.decomposition import PCA
from sklearn.cluster import *
from collections import Counter
from tqdm.notebook import tqdm
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')
nltk.download('stopwords')
stops = set(stopwords.words('russian'))

def normalize(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(text) if word]
    return words

def tokenize(text):
    return token.tokenize(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## 1. Значения слов

Словарь, из которого будем извлекать значения слов - Wiktionary.



In [4]:
lexemes = ['отражать', 'идти', 'слух', 'кисть', 'звезда']

In [5]:
parser = WiktionaryParser()

def get_definitions():
    df_definitions = pd.DataFrame(columns=['word', 'context'])
    definitions = []
    for lexeme in lexemes:
        parsed_response = parser.make_request(lexeme)
        for item in parsed_response['definitions']:
            definition = lexeme + ': ' + item['value'] + '. Пример: ' + item['example']
            df_definitions.loc[len(df_definitions.index)] = [lexeme, definition]
    return df_definitions

In [6]:
df_definitions = get_definitions()
df_definitions

,word,context
0,отражать,"отражать: отбивать или отклонять преградой, от..."
1,отражать,"отражать: перен. отвечая на обвинения, нападки..."
2,отражать,отражать: отбрасывать в обратном направлении с...
3,отражать,"отражать: воспроизводить изображение, какой-ли..."
4,отражать,"отражать: передавать, воспроизводить в художес..."
5,отражать,"отражать: перен. соответствовать чему-либо, де..."
6,идти,"идти: двигаться, передвигаться, ступая ногами...."
7,идти,идти: перен. совершать поступательное движение...
8,идти,"идти: перен. собираться, отправляться, направл..."
9,идти,идти: перен. действовать на основе принятого р...


## 2. Извлечение контекстов из корпуса

Выбираем Генеральный интернет-корпус русского языка (ГИКРЯ).

In [ ]:
!wget https://github.com/dialogue-evaluation/morphoRuEval-2017/raw/master/GIKRYA_texts_new.zip
!unzip GIKRYA_texts_new.zip
!rm GIKRYA_texts_new.zip

In [8]:
from corus import load_morphoru_gicrya

def get_contexts():
    df_corpus = pd.DataFrame(columns=['word', 'context', 'lemmas'])
    path = 'gikrya_new_train.out'
    records = load_morphoru_gicrya(path)
    for record in tqdm(records):
        add = False
        lexeme = ''
        context = []
        lemmas = []
        for token in record.tokens:
            if token.lemma in lexemes:
                add = True
                lexeme = token.lemma
            context.append(token.text)
            lemmas.append(token.lemma)
        if add:
            df_corpus.loc[len(df_corpus.index)] = [lexeme, ' '.join(context), lemmas]
    return df_corpus

In [9]:
df_corpus = get_contexts()

0it [00:00, ?it/s]

In [10]:
df_corpus.head(10)

,word,context,lemmas
0,звезда,"Перед глазами вспыхнули звезды , а тело скорчи...","[перед, глаз, вспыхнуть, звезда, ,, а, тело, с..."
1,звезда,Света звезд и яркой в это время года луны впол...,"[свет, звезда, и, яркий, в, этот, время, год, ..."
2,слух,"Его слух , наверное , еще не восстановился пол...","[его, слух, ,, наверное, ,, ещё, не, восстанов..."
3,слух,Слухам о ваших пикантных отношениях с моей пле...,"[слух, о, ваш, пикантный, отношение, с, мой, п..."
4,звезда,"Он награжден рядом государственных наград , в ...","[он, награждённый, ряд, государственный, награ..."
5,идти,"От рукавиц шел уже не пар , а дым .","[от, рукавица, идти, уже, не, пар, ,, а, дым, .]"
6,идти,"Сарториус же утверждал , что раны должны дышат...","[Сарториус, же, утверждать, ,, что, рана, долж..."
7,идти,И запах шел из кухни .,"[и, запах, идти, из, кухня, .]"
8,идти,. лень было куда-то идти - ехать .,"[., лень, быть, куда-то, идти, -, ехать, .]"
9,идти,"Передовой дозор и , похоже , что идут параллел...","[передовой, дозор, и, ,, похожий, ,, что, идти..."


## 3. WSI

### Adagram

In [ ]:
!curl "https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib" > /content/drive/MyDrive/all.a010.p10.d300.w5.m100.nonorm.slim.joblib

In [12]:
model_adagram = adagram.VectorModel.load('/content/drive/MyDrive/all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [13]:
def disambiguate(model, word, lemmatized_context):
    probs = model.disambiguate(word, lemmatized_context)
    return 1 + probs.argmax()

In [14]:
def apply_adagram(df):
    df['predict_sense_id_adagram'] = [disambiguate(model_adagram, word, context)
                              for word, context in tqdm(zip(df['word'], df['lemmas']), total=len(df))]
    return df

In [15]:
df_corpus = apply_adagram(df_corpus)

  0%|          | 0/730 [00:00<?, ?it/s]

### ELMo

In [ ]:
!wget http://vectors.nlpl.eu/repository/20/196.zip
!unzip 196.zip -d 196

In [17]:
model_elmo = ElmoModel()
model_elmo.load('196')

'The model is now loaded.'

In [18]:
def get_elmo_vectors(word, lemmatized_contexts, model):
    all_vectors = model.get_elmo_vectors(lemmatized_contexts)
    word_vecs = []
    for i in range(len(lemmatized_contexts)):
        try:
            word_vecs.append(all_vectors[i][lemmatized_contexts[i].index(word)])
        except ValueError:  # если нормализация накосячила и лемму не найти
            continue
    return word_vecs

In [19]:
def apply_elmo_fit(df):
    clusters = dict()
    df['predict_sense_id_elmo'] = ''
    grouped_df = df.groupby('word')[['word', 'lemmas']]
    for word, _ in grouped_df:
        lemmas = grouped_df.get_group(word)['lemmas'].to_list()
        X = get_elmo_vectors(word, lemmas, model_elmo)
        if len(X) != len(lemmas):
            continue
        cluster = AffinityPropagation(damping=0.5)
        cluster.fit(X)
        clusters[word] = cluster
        labels = np.array(cluster.labels_) + 1
        df.loc[df['word'] == word, 'predict_sense_id_elmo'] = labels

    return clusters, df

In [20]:
clusters, df_corpus = apply_elmo_fit(df_corpus)
df_corpus

,word,context,lemmas,predict_sense_id_adagram,predict_sense_id_elmo
0,звезда,"Перед глазами вспыхнули звезды , а тело скорчи...","[перед, глаз, вспыхнуть, звезда, ,, а, тело, с...",4,11
1,звезда,Света звезд и яркой в это время года луны впол...,"[свет, звезда, и, яркий, в, этот, время, год, ...",4,10
2,слух,"Его слух , наверное , еще не восстановился пол...","[его, слух, ,, наверное, ,, ещё, не, восстанов...",3,1
3,слух,Слухам о ваших пикантных отношениях с моей пле...,"[слух, о, ваш, пикантный, отношение, с, мой, п...",2,5
4,звезда,"Он награжден рядом государственных наград , в ...","[он, награждённый, ряд, государственный, награ...",2,1
...,...,...,...,...,...
725,слух,Будучи обладателем красивого голоса и тонкого ...,"[быть, обладатель, красивый, голос, и, тонкий,...",1,6
726,идти,"идет он , во всю идет <emo>","[идти, он, ,, во, весь, идти, <emo>]",2,39
727,идти,"Я шла весь день по жаре , много потела .","[я, идти, весь, день, по, жара, ,, много, поте...",4,16
728,идти,"Тогда я буду знать , куда идти !","[тогда, я, быть, знать, ,, куда, идти, !]",5,15


## 4. WSI для словарных значений

In [21]:
token = RegexpTokenizer('\w+')
morph = MorphAnalyzer()

def tokenize(text):
    return token.tokenize(text)

def normalize(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(text) if word]
    return words

In [22]:
def apply_elmo_predict(clusters, df):
    df['predict_sense_id_elmo'] = ''
    grouped_df = df.groupby('word')[['word', 'lemmas']]
    for word, _ in grouped_df:
        lemmas = grouped_df.get_group(word)['lemmas'].to_list()
        X = get_elmo_vectors(word, lemmas, model_elmo)
        if len(X) != len(lemmas):
            continue
        predictions = clusters[word].predict(X)
        labels = np.array(predictions) + 1
        df.loc[df['word'] == word, 'predict_sense_id_elmo'] = labels

    return df

In [23]:
df_definitions['lemmas'] = df_definitions['context'].apply(normalize)
df_definitions = apply_adagram(df_definitions)
df_definitions = apply_elmo_predict(clusters, df_definitions)

  0%|          | 0/37 [00:00<?, ?it/s]

In [24]:
df_definitions

,word,context,lemmas,predict_sense_id_adagram,predict_sense_id_elmo
0,отражать,"отражать: отбивать или отклонять преградой, от...","[отражать, отбивать, или, отклонять, преграда,...",3,2
1,отражать,"отражать: перен. отвечая на обвинения, нападки...","[отражать, перена, отвечать, на, обвинение, на...",5,2
2,отражать,отражать: отбрасывать в обратном направлении с...,"[отражать, отбрасывать, в, обратный, направлен...",1,2
3,отражать,"отражать: воспроизводить изображение, какой-ли...","[отражать, воспроизводить, изображение, какой,...",3,3
4,отражать,"отражать: передавать, воспроизводить в художес...","[отражать, передавать, воспроизводить, в, худо...",5,3
5,отражать,"отражать: перен. соответствовать чему-либо, де...","[отражать, перена, соответствовать, что, либо,...",5,2
6,идти,"идти: двигаться, передвигаться, ступая ногами....","[идти, двигаться, передвигаться, ступать, нога...",2,39
7,идти,идти: перен. совершать поступательное движение...,"[идти, перена, совершать, поступательный, движ...",1,39
8,идти,"идти: перен. собираться, отправляться, направл...","[идти, перена, собираться, отправляться, напра...",5,39
9,идти,идти: перен. действовать на основе принятого р...,"[идти, перена, действовать, на, основа, принят...",1,32


## 5. Выбор значений и разметка

In [25]:
df_definitions = df_definitions.iloc[[0, 4, 6, 9, 20, 22, 25, 27, 31, 33]]

Берем следующие значения:

In [26]:
df_definitions

,word,context,lemmas,predict_sense_id_adagram,predict_sense_id_elmo
0,отражать,"отражать: отбивать или отклонять преградой, от...","[отражать, отбивать, или, отклонять, преграда,...",3,2
4,отражать,"отражать: передавать, воспроизводить в художес...","[отражать, передавать, воспроизводить, в, худо...",5,3
6,идти,"идти: двигаться, передвигаться, ступая ногами....","[идти, двигаться, передвигаться, ступать, нога...",2,39
9,идти,идти: перен. действовать на основе принятого р...,"[идти, перена, действовать, на, основа, принят...",1,32
20,слух,"слух: одно из чувств живого существа, способно...","[слух, один, из, чувство, живой, существо, спо...",3,1
22,слух,"слух: перен. то же, что сплетня изустно распро...","[слух, перена, то, же, что, сплетня, изустно, ...",2,5
25,кисть,кисть: инструмент для нанесения на поверхность...,"[кисть, инструмент, для, нанесение, на, поверх...",1,5
27,кисть,кисть: анат. часть руки человека или передней ...,"[кисть, анат, часть, рука, человек, или, перед...",3,1
31,звезда,"звезда: вещь, предмет в форме звезды. Пример: ...","[звезда, вещь, предмет, в, форма, звезда, прим...",4,10
33,звезда,"звезда: перен. известный артист, знаменитость ...","[звезда, перена, известный, артист, знаменитос...",3,8


In [27]:
df_adagram = df_corpus.merge(df_definitions[['word', 'predict_sense_id_adagram']])
df_adagram.to_csv('adagram.csv', index=False)

In [28]:
df_elmo = df_corpus.merge(df_definitions[['word', 'predict_sense_id_elmo']])
df_elmo.to_csv('elmo.csv', index=False)

## 6. Подсчет accuracy

In [32]:
adagram_annotated = pd.read_csv('adagram_annotated.csv')
adagram_annotated.head(10)

,word,context,lemmas,predict_sense_id_adagram,adagram_true
0,звезда,"Перед глазами вспыхнули звезды , а тело скорчи...","['перед', 'глаз', 'вспыхнуть', 'звезда', ',', ...",4,1
1,звезда,Света звезд и яркой в это время года луны впол...,"['свет', 'звезда', 'и', 'яркий', 'в', 'этот', ...",4,0
2,звезда,"Я долго всматривалась в небо , стараясь увидет...","['я', 'долго', 'всматриваться', 'в', 'небо', '...",4,0
3,звезда,"Вершины гор , освещенные первыми звездами , бы...","['вершина', 'гора', ',', 'освещённый', 'первый...",4,0
4,звезда,""" Это звезда , с неба упавшая — , "" кратко объ...","['""', 'это', 'звезда', ',', 'с', 'небо', 'упав...",4,0
5,слух,"Его слух , наверное , еще не восстановился пол...","['его', 'слух', ',', 'наверное', ',', 'ещё', '...",3,1
6,слух,"Включила , послушала — и даже гитара на слух н...","['включить', ',', 'послушать', '—', 'и', 'даже...",3,1
7,слух,Его усиленный акустической системой слух пронз...,"['его', 'усиленный', 'акустический', 'система'...",3,1
8,слух,Каждый прислушивался к нарастающему вою снаряд...,"['каждый', 'прислушиваться', 'к', 'нарастающий...",3,1
9,слух,способствует заживлению и восстановлению бараб...,"['способствовать', 'заживление', 'и', 'восстан...",3,1


In [33]:
elmo_annotated = pd.read_csv('elmo_annotated.csv')
elmo_annotated.head(10)

,word,context,lemmas,predict_sense_id_elmo,elmo_true
0,звезда,Света звезд и яркой в это время года луны впол...,"['свет', 'звезда', 'и', 'яркий', 'в', 'этот', ...",10,0
1,звезда,"Мамонт разжал зубы , крикнул и почудилось , зв...","['мамонт', 'разжать', 'зуб', ',', 'крикнуть', ...",10,0
2,звезда,"Лучи звезды вспыхнули , и он почувствовал как ...","['луч', 'звезда', 'вспыхнуть', ',', 'и', 'он',...",10,0
3,звезда,"Жёлтая и оранжевая звезды составляют центр , а...","['жёлтый', 'и', 'оранжевый', 'звезда', 'состав...",10,0
4,звезда,Первые звезды зажглись на небе .,"['первый', 'звезда', 'зажечься', 'на', 'небо',...",10,0
5,слух,"Его слух , наверное , еще не восстановился пол...","['его', 'слух', ',', 'наверное', ',', 'ещё', '...",1,1
6,слух,Его усиленный акустической системой слух пронз...,"['его', 'усиленный', 'акустический', 'система'...",1,1
7,слух,"А ты не боишься , что эта лавина слухов будет ...","['а', 'ты', 'не', 'бояться', ',', 'что', 'этот...",1,0
8,слух,Так это не сторонники майдана слухи распускают...,"['так', 'это', 'не', 'сторонник', 'майдан', 'с...",1,0
9,слух,И этот слух вскоре подтвердился официально .,"['и', 'этот', 'слух', 'вскоре', 'подтвердиться...",1,0


In [34]:
adagram_true = adagram_annotated['adagram_true']
elmo_true = elmo_annotated['elmo_true']
print('Adagram accuracy:', '{:.2%}'.format(sum(adagram_true) / len(adagram_true)))
print('Elmo accuracy:', '{:.2%}'.format(sum(elmo_true) / len(elmo_true)))

Adagram accuracy: 56.25%
Elmo accuracy: 47.83%


## 7. Сравнение

- И у Adagram, и у Elmo + Affinity Clustering низкое значение accuracy, причем у Elmo + Affinity Clustering ниже (скорее всего, за счет того, что у кластеризации специально не подбирались гиперпараметры)

- И у Adagram, и у Elmo + Affinity Clustering тенденция выделять в один кластер либо все правильные значения (оба значения слова "слух" у Adagram; "идти" как передвигаться ногами" и "кисть" как "кисть руки" у Elmo), либо все неправильные ("идти" как "действовать на основе принятого решения" у обоих; "кисть" как "кисть руки" у Adagram; "отражать" как "отбрасывать в обратном направлении свет/звук" и "звезда" как "предмет в форме звезды" у Elmo).

- Affinity Clustering выделяет большое количество кластеров, большинство из которых оказываются пустыми, в то же время помещая значения слов только в два разных кластера. Оценивая качество модели только относительно одного значения, попавшего в кластер, я игнорировал другие значения, которые ему принадлежат, и за счет этого у Elmo низкая accuracy (что рационально, потому что в идеальном случае разные значения должны определяться в разные кластеры)

- Elmo лучше справляется с определением предложений со словом "кисть" как "кисть руки" в правильный кластер:
```
Ствол револьвера, кисть руки, держащей оружие и сама рука должны составлять одну прямую линию.
Задержка соперника представляет собой препятствование его движению с помощью кистей рук, рук или корпуса.
Игра рукой подразумевает намеренное действие игрока, касающегося мяча кистью руки или рукой.
```
